This notebook:
1. reads in .csv files with specs and timeseries power that came from [Get_ERCOT_EIA_Data.ipynb](Get_ERCOT_EIA_Data.ipynb)
2. converts specs from EIA and ERCOT into modeling parameters
3. makes forecasts 

It is based on https://github.com/williamhobbs/PVPMC_2025/blob/main/Process_Specs_and_Data.ipynb.

For now, I'm leaving out the parameter optimization (gcr, loss factor, gamma pdc) for this forecasting project.

Note on IFS: 20240305 appears to be the first day that IFS has the parameters we need, so queries will start on the following Sunday (202240310)

In [1]:
import pandas as pd
import glob
import os
import datetime
# import pvlib
from pv_model import model_pv_power
from forecast_solar import get_solar_forecast_fast
import matplotlib.pyplot as plt
# import scipy
import numpy as np

Read in all the specs:

In [2]:
folder = 'output_specs' 
filelist = glob.glob(os.path.join(folder , '*_specs.csv'))
df = pd.concat((pd.read_csv(f) for f in filelist), ignore_index=True)

# replace empty strings with NaN
df = df.replace(' ', np.nan)

num_plants = len(df)

C:\Users\willh\AppData\Local\Temp\ipykernel_22584\3833113069.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(' ', np.nan)


Since EIA nameplates seem to be off (sometimes it includes multiple units, other times I'm not sure what's happening), let's normalize EIA AC and DC nameplate based on the ratio of EIA AC to CDR AC nameplates. 

In [3]:
ratios = df['cdr file nameplate ac mw'] / df['eia solar Nameplate Capacity (MW)']
df['eia nameplate ac'] = df['eia nameplate ac']*ratios
df['eia solar Nameplate Capacity (MW)'] = df['eia solar Nameplate Capacity (MW)'] * ratios
df['eia solar DC Net Capacity (MW)'] = df['eia solar DC Net Capacity (MW)'] * ratios

Function to convert EIA specs to pv_model parameters:

In [4]:
def eia_specs_to_pv_model_params(plant_data):

    # basics
    latitude = plant_data['Latitude']
    longitude = plant_data['Longitude']
    nameplate_dc = plant_data['DC Net Capacity (MW)']
    nameplate_ac = plant_data['Nameplate Capacity (MW)']
    
    # commercial operation date
    cod = datetime.datetime(
        int(plant_data['Operating Year']),
        int(plant_data['Operating Month']),
        1, # assume first day of the month
        ) 

    # mounting type
    if plant_data['Single-Axis Tracking?'] == 'Y':
        mount_type = 'single-axis'
    elif plant_data['Fixed Tilt?'] == 'Y':
        mount_type = 'fixed'
    else:
        raise ValueError('plant does not appear to be fixed or single-axis')

    # shade loss.  Use linear for cdte, for crystalline we don't have info about twin cells/modules,
    # so assume twin (half-cut) after a given date
    twin_module_start_date = pd.to_datetime('2020-01-01') # TODO: needs to be verified, maybe in ITRPV 2020 or 2021?
    if plant_data['Thin-Film (CdTe)?'] == 'Y':
        shade_loss_model = 'linear'
        cell_type = 'thin-film_cdte' # for spectral correction
        n_cells_up = 1
    elif (
        plant_data['Thin-Film (A-Si)?'] == 'Y' or
        plant_data['Thin-Film (CIGS)?'] == 'Y' or
        plant_data[ 'Thin-Film (Other)?'] == 'Y'
    ):
        shade_loss_model = 'linear'
        n_cells_up = 1
    elif plant_data['Crystalline Silicon?'] == 'Y':
        cell_type = 'crystalline'
        if cod > twin_module_start_date:
            shade_loss_model = 'non-linear_simple_twin_module'
            n_cells_up = 24
        else:
            shade_loss_model = 'non-linear_simple'
            n_cells_up = 12
        
    # bifacial
    if plant_data['Bifacial?'] == 'Y':
        bifacial = True
        bifaciality_factor = 0.8 # TODO: consider changing this based on COD
    else:
        bifacial = False
        bifaciality_factor = pd.NA

    # backtracking
    if (
        mount_type == 'single-axis' and
        shade_loss_model == 'linear'
    ):
        backtrack = False
    else:
        backtrack = True

    # gcr 
    if mount_type == 'fixed':
        gcr = 0.55
    elif mount_type == 'single-axis' and cell_type == 'crystalline':
        gcr = 0.35
    elif mount_type == 'single-axis':
        gcr = 0.40

    # modules up each row 
    if mount_type == 'fixed' and cell_type == 'crystalline':
        row_side_num_mods = 2
    else:
        # assume crystalline trackers are 1P,
        # doesn't matter for linear shade loss configurations
        row_side_num_mods = 1

    # tilt and azimuth
    if mount_type == 'fixed':
        # assume tilt and azimuth are correct for fixed systems
        fixed_azimuth = plant_data['Azimuth Angle']
        fixed_tilt = plant_data['Tilt Angle']
        axis_azimuth = pd.NA
        axis_tilt = pd.NA
    elif mount_type == 'single-axis':
        # sometimes EIA has the wrong azimuth for trackers, e.g., they enter tilt
        if plant_data['Azimuth Angle'] > 90 and plant_data['Azimuth Angle'] < 270:
            axis_azimuth = plant_data['Azimuth Angle']
        else:
            axis_azimuth = 180
        # sometimes EIA has the wrong tilt for trackers, e.g., they enter max rotation angle
        if isinstance(plant_data['Tilt Angle'],str):
            axis_tilt = 0
        elif plant_data['Tilt Angle'] < 15:
            axis_tilt = plant_data['Tilt Angle']
        else:
            axis_tilt = 0
        fixed_azimuth = pd.NA
        fixed_tilt = pd.NA

    # temperature coefficient
    # TODO: consider changing this based on COD
    if cell_type == 'crystalline':
        gamma_pdc = -0.0035
    else:
        gamma_pdc = -0.0025

    # dc derate
    dc_loss_fraction_base = 0.15
    annual_degradation = 0.005 # assume 0.5%/yr PLR
    years_oper = (pd.Timestamp.now() - cod).days / 365.25 # TODO: make this based analysis time, not time right now
    degr_loss = years_oper * annual_degradation
    dc_loss_fraction = 1 - ((1 - dc_loss_fraction_base) * (1 - degr_loss))

    plant_data_new = {
        'latitude': latitude,
        'longitude': longitude,
        'nameplate_dc': nameplate_dc,
        'nameplate_ac': nameplate_ac,
        'mount_type': mount_type,
        'cell_type': cell_type,
        'shade_loss_model': shade_loss_model,
        'bifacial': bifacial,
        'bifaciality_factor': bifaciality_factor,
        'backtrack': backtrack,
        'gcr': gcr,
        'row_side_num_mods': row_side_num_mods,
        'fixed_azimuth': fixed_azimuth,
        'fixed_tilt': fixed_tilt,
        'axis_azimuth': axis_azimuth,
        'axis_tilt': axis_tilt,
        'gamma_pdc': gamma_pdc,
        'dc_loss_fraction': dc_loss_fraction,
        'n_cells_up': n_cells_up,
        'cod': cod,
    }

    return plant_data_new

And code to loop through each plant, convert the specs, convert them:

In [5]:
agg_specs_orig = {}

for plant_number in range(num_plants):
    ## Get plant metadata ready
    # =====================================================
    # select a row from dataframe and convert to dictionary
    # plant_data = df.iloc[plant_number].dropna().to_dict()
    plant_data = df.iloc[plant_number].to_dict()

    # select keys that start with 'eia_solar'
    prefix = 'eia solar'
    plant_data = {key: val for key, val in plant_data.items()
            if key.startswith(prefix)}

    # remove prefix
    plant_data = {key.lstrip(prefix):value for key, value in plant_data.items()}

    # convert EIA parameters to pv_model parameters
    plant_data_new = eia_specs_to_pv_model_params(plant_data)
   
    # outputs
    agg_specs_orig[plant_number] = plant_data_new

    print('done with plant ' + str(plant_number))

done with plant 0
done with plant 1
done with plant 2
done with plant 3
done with plant 4
done with plant 5
done with plant 6
done with plant 7
done with plant 8
done with plant 9
done with plant 10
done with plant 11
done with plant 12
done with plant 13
done with plant 14
done with plant 15
done with plant 16
done with plant 17
done with plant 18
done with plant 19


In [6]:
agg_specs_orig_df = pd.DataFrame(agg_specs_orig)

In [7]:
agg_specs_orig_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
latitude,32.811041,31.719983,33.021601,31.888,31.255,33.404486,30.4214,28.883196,31.435932,31.030996,32.104125,33.467803,32.4744,32.460054,32.533273,34.380532,26.090103,33.261505,29.271667,29.242304
longitude,-99.91749,-104.442292,-99.61463,-100.825,-102.272,-96.082472,-97.4614,-99.178605,-99.796613,-102.488209,-100.162375,-95.370856,-95.7156,-102.672809,-96.428985,-100.099625,-97.798453,-97.238057,-98.444722,-95.658144
nameplate_dc,131.5944,256.362179,94.507156,130.434783,234.0,170.854369,186.0,176.0,319.0,172.362353,143.08,260.0,82.098305,133.931,182.009816,165.306333,184.861111,191.275636,53.936118,162.4
nameplate_ac,100.8,188.2,74.9,100.0,180.0,125.7,144.0,132.4,250.0,126.3,102.2,198.5,59.8,100.7,147.6,121.4,137.5,148.8,39.2,120.0
mount_type,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis,single-axis
cell_type,crystalline,thin-film_cdte,crystalline,thin-film_cdte,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,crystalline,thin-film_cdte
shade_loss_model,non-linear_simple_twin_module,linear,non-linear_simple_twin_module,linear,non-linear_simple,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple,non-linear_simple_twin_module,non-linear_simple,non-linear_simple_twin_module,non-linear_simple_twin_module,non-linear_simple,linear
bifacial,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
bifaciality_factor,<NA>,<NA>,0.8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0.8,<NA>,<NA>,<NA>,<NA>,<NA>
backtrack,True,False,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False


Write the specs to a pickle for use later:

In [8]:
agg_specs_orig_df.to_pickle('output_specs/agg_specs_orig_df.pkl')

Now, let's make forecasts.

In [9]:
latitudes = agg_specs_orig_df.loc['latitude'].to_list()
longitudes = agg_specs_orig_df.loc['longitude'].to_list()

dates = pd.date_range(start = '2024-03-10 12:00', end = '2025-03-02 12:00', freq = '7D')
# dates = pd.date_range(start = '2024-03-10 12:00', end = '2024-03-18 12:00', freq = '7D')
run_length = 168
lead_time_to_start = 24

In [ ]:
dfs = []
for date in dates:
    resource_data_temp = get_solar_forecast_fast(
        latitude=latitudes,
        longitude=longitudes,
        init_date=date,
        run_length=run_length,
        lead_time_to_start=lead_time_to_start,
        model='gfs',
        attempts=4,
    )
    # # add init_time as a column, then convert to index
    # resource_data_temp['init_time'] = date
    # resource_data_temp.set_index(['init_time', resource_data_temp.index], inplace=True)
    dfs.append(resource_data_temp)

resource_data_gfs = pd.concat(dfs)
resource_data_gfs['albedo'] = 0.2 # fill in albedo data

c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\core.py:1112: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()

In [26]:
resource_data_gfs[resource_data_gfs['point']==0].head()

point   temp_air  wind_speed  \
init_time           valid_time                                                
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00      0   8.618383    4.222985   
                    2024-03-11 13:30:00+00:00      0   9.007821    4.298540   
                    2024-03-11 14:30:00+00:00      0  11.289124    5.403147   
                    2024-03-11 15:30:00+00:00      0  14.098417    6.938874   
                    2024-03-11 16:30:00+00:00      0  16.250700    7.433364   

                                                ghi_csi         ghi  \
init_time           valid_time                                        
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00  5.738516    0.000000   
                    2024-03-11 13:30:00+00:00  0.873521   68.223819   
                    2024-03-11 14:30:00+00:00  0.969681  288.712569   
                    2024-03-11 15:30:00+00:00  1.013909  505.125819   
                    2024-03-11 16:30:00+00:00  1.038190  684.714747   

                                                      dni         dhi  \
init_time           valid_time                                          
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00    0.000000    0.000000   
                    2024-03-11 13:30:00+00:00  119.789171   54.396269   
                    2024-03-11 14:30:00+00:00  580.793574  100.316410   
                    2024-03-11 15:30:00+00:00  774.633604  110.802698   
                    2024-03-11 16:30:00+00:00  854.289140  123.571776   

                                                ghi_clear  albedo  
init_time           valid_time                                     
2024-03-10 12:00:00 2024-03-11 12:30:00+00:00    0.000000     0.2  
                    2024-03-11 13:30:00+00:00   78.102100     0.2  
                    2024-03-11 14:30:00+00:00  297.739588     0.2  
                    2024-03-11 15:30:00+00:00  498.196465     0.2  
                    2024-03-11 16:30:00+00:00  659.527318     0.2

In [ ]:
resource_data_gfs.to_pickle('forecasts/resource_data_gfs.pkl')
resource_data_gfs.to_csv('forecasts/resource_data_gfs.csv')

In [ ]:
dfs = []
for date in dates:
    resource_data_temp = get_solar_forecast_fast(
        latitude=latitudes,
        longitude=longitudes,
        init_date=date,
        run_length=run_length,
        lead_time_to_start=lead_time_to_start,
        model='ifs',
        attempts=4,
    )
    # # add init_time as a column, then convert to index
    # resource_data_temp['init_time'] = date
    # resource_data_temp.set_index(['init_time', resource_data_temp.index], inplace=True)
    dfs.append(resource_data_temp)

resource_data_ifs = pd.concat(dfs)
resource_data_ifs['albedo'] = 0.2 # fill in albedo data

c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")


👨🏻‍🏭 Created directory: [C:\Users\willh\data\ifs\20240317]


c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")


👨🏻‍🏭 Created directory: [C:\Users\willh\data\ifs\20240324]


c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
Exception has occured : ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025

...

c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")
c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")


👨🏻‍🏭 Created directory: [C:\Users\willh\data\ifs\20250223]


c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")


👨🏻‍🏭 Created directory: [C:\Users\willh\data\ifs\20250302]


c:\Users\willh\miniforge3\envs\pvsc_forecaster_2025\Lib\site-packages\herbie\accessors.py:303: UserWarning: `with_wind()` did not do anything.
  warnings.warn("`with_wind()` did not do anything.")


In [ ]:
resource_data_ifs[resource_data_ifs['point']==0].head()

,point,temp_air,wind_speed,ghi_csi,ghi,dni,dhi,ghi_clear,albedo
valid_time,,,,,,,,,
2024-03-11 12:30:00+00:00,0,7.796638,2.774506,0.923495,0.000000,0.000000,0.000000,0.000000,0.2
2024-03-11 13:30:00+00:00,0,9.208178,3.174082,0.923495,72.126869,153.411860,54.418171,78.102100,0.2
2024-03-11 14:30:00+00:00,0,10.619718,3.573658,0.923495,274.960907,498.324407,113.315861,297.739588,0.2
2024-03-11 15:30:00+00:00,0,12.428915,4.061708,1.016497,506.415199,779.028187,109.855039,498.196465,0.2
2024-03-11 16:30:00+00:00,0,14.649252,4.641232,1.016497,670.407522,821.735906,130.647263,659.527318,0.2


In [ ]:
resource_data_ifs.to_pickle('forecasts/resource_data_ifs.pkl')
resource_data_ifs.to_csv('forecasts/resource_data_ifs.csv')

Run this if needed to read the CSVs back in:

In [10]:
resource_data_gfs = pd.read_csv('forecasts/resource_data_gfs.csv', parse_dates=['valid_time'], index_col='valid_time')
resource_data_ifs = pd.read_csv('forecasts/resource_data_ifs.csv', parse_dates=['valid_time'], index_col='valid_time')

Now, let's model power using the resource forecasts to get power forecasts:

In [47]:
# empty lists
gfs_temp_list = []
ifs_temp_list = []

for plant_number in range(num_plants):
    # pull plant data
    plant_data = agg_specs_orig_df[plant_number].to_dict()

    # get the resource forecast for the plant
    resource_forecast_gfs = resource_data_gfs[resource_data_gfs['point']==plant_number]
    resource_forecast_ifs = resource_data_ifs[resource_data_ifs['point']==plant_number]

    # model power
    power_fcasts_gfs, _ = model_pv_power(resource_forecast_gfs, **plant_data)
    power_fcasts_ifs, _ = model_pv_power(resource_forecast_ifs, **plant_data)

    power_fcasts_gfs.name = 'power'
    power_fcasts_ifs.name = 'power'

    # convert to dataframe and add 'point' (plant number)
    power_fcasts_gfs = power_fcasts_gfs.to_frame()
    power_fcasts_ifs = power_fcasts_ifs.to_frame()
    power_fcasts_gfs['point'] = plant_number
    power_fcasts_ifs['point'] = plant_number

    gfs_temp_list.append(power_fcasts_gfs)
    ifs_temp_list.append(power_fcasts_ifs)

power_fcasts_gfs_all = pd.concat(gfs_temp_list)
power_fcasts_ifs_all = pd.concat(ifs_temp_list)

c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['poa_modeled'] = (total_irrad.poa_diffuse +
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['poa_modeled'] = \
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:400: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

pvlib.bifacial.infinite_sheds does not currently accept the perez-driesse model.
using haydavies instead.


c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data.fillna(0, inplace=True)
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['poa_modeled'] = (total_irrad.poa_diffuse +
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

pvlib.bifacial.infinite_sheds does not currently accept the perez-driesse model.
using haydavies instead.


c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data.fillna(0, inplace=True)
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:347: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['precipitable_water'] = 1
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

pvlib.bifacial.infinite_sheds does not currently accept the perez-driesse model.
using haydavies instead.


c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data.fillna(0, inplace=True)
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['poa_modeled'] = (total_irrad.poa_diffuse +
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

pvlib.bifacial.infinite_sheds does not currently accept the perez-driesse model.
using haydavies instead.


c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:481: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data.fillna(0, inplace=True)
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:358: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resource_data['poa_modeled'] = (total_irrad.poa_diffuse +
c:\Users\willh\Documents\Python Scripts\daily_energy_forecaster_2\pv_model.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

And write resulting dataframes to CSV and pickle files.

In [48]:
power_fcasts_gfs_all.to_pickle('forecasts/power_fcasts_gfs_all.pkl')
power_fcasts_ifs_all.to_pickle('forecasts/power_fcasts_ifs_all.pkl')

power_fcasts_gfs_all.to_csv('forecasts/power_fcasts_gfs_all.csv')
power_fcasts_ifs_all.to_csv('forecasts/power_fcasts_ifs_all.csv')